# PROJET PYTHON POUR LA DATA SCIENCE 2022

# THEME : ANALYSE DES AVIS DES STAGIAIRES ET EMPLOYES DE SOCIETE GENERALE TRANSMIS VIA LA PLATEFORME INDEED.COM ENTRE 2012 ET 2022  

## Auteurs:
* TALEB AHMED Raja
* MAGAJIE WAMSA Berthe Magella
* RENE Léo

## Scrapping des commentaires des employés et stagiaires de Société Générale sur le site Indeed.com

Le but de cette partie est de récupérer par webscrapping les données devant servir à notre analyse. Il s'agit notamment des retours d'expérience et commentaires des employés et stagiaires de SOCIETE GENERALE transmis sur la plateforme Indeed.com entre 2012 et 2022.

In [1]:
# Installer selenium
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 27.6 MB/s eta 0:00:00


In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Scrapping des données

Pour récupérer nos données sur le site Indeed, nous utilisons le package Selenium, étant donné que le site web n'est pas statique. Le pilote Web utilisé est Chrome.

In [37]:
# Ouvrir le site Indeed en utilisant chromedriver
driver=webdriver.Chrome('chromedriver')
driver.get('https://fr.indeed.com/cmp/Soci%C3%A9t%C3%A9-G%C3%A9n%C3%A9rale-1/reviews')

WebDriverException: Message: Service chromedriver unexpectedly exited. Status code was: 127


In [29]:
from selenium.webdriver.remote.errorhandler import NoSuchElementException
fichier = open("sg.txt", "w", encoding="utf-8")
i = 1

suivant_xpath = '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/nav/ul/li/a'

while(True):
    xpath = '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/div[' + str(i) + ']'
    try :
        metier = driver.find_element(By.XPATH, xpath)
        fichier.write(metier.text+ 2*'\n')
        
        sep = 10*'#'
        fichier.write(sep + 2*'\n')
        i=i+1
        
    except NoSuchElementException : 
        time.sleep(60)
        suivant = driver.find_element(By.XPATH, suivant_xpath)
        suivant_xpath = '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/nav/ul/li[2]/a'
        suivant.click()
        i=1


NameError: name 'driver' is not defined

In [ ]:
fichier.close()

# Création et mise en forme de la base de données 

Dans cette partie, un dataframe est constitué à partir des données scrappées et stockées au préalable dans le fichier sg.txt ; la base de données ainsi construite comporte 1852 lignes/observations pour 6 variables que sont **'rating'** qui renferme les notes attribuées (sur 5), **'contract'** pour le type de contrat(Stage, cdd, cdi), **'job'** pour le poste occupé, **'city'** pour le lieu de travail (la ville) , **'date'** pour la date de publication du commentaire et **'comment'** pour les commentaires.

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [55]:
fichier = open('sg.txt', 'r', encoding="utf-8")
text = fichier.read()
parts = text.split('\n##########\n\n')

In [63]:
errors = {
    'rating': [],
    'job_city_date': []        
}

df = pd.DataFrame(columns= ['rating', 'contract', 'job', 'city', 'date', 'comment'])

for avis in parts:
    new_row = {}
    avis_par_ligne = avis.split('\n')
    i = 0
    
    rating = avis_par_ligne[i]
    # convertir par exemple '3,0' à 3.0
    rating = rating.replace(',', '.')
    try:
        # si la première ligne correspond à un classement, on l'enregistre
        rating = float(rating)

        # passage à la ligne suivante
        i+=1

    except ValueError:
        errors['rating'].append(avis)
        continue

    contract_or_title = avis_par_ligne[i]
    i+=1

    job_city_date = avis_par_ligne[i]

    try :
        job, city, date = job_city_date.split(' - ')
        i+=1

    except ValueError : 

        errors['job_city_date'].append(avis)
        continue

    comment = ''
    while avis_par_ligne[i] in ['Avis à la une', "L'avis le plus utile, sélectionné par Indeed"]:
        i+=1
 


    while i<len(avis_par_ligne) and  avis_par_ligne[i]!= 'Avez-vous trouvé cet avis utile ?':
        
        comment += avis_par_ligne[i] + '\n'
        i+=1
        
    new_row = {
        'rating': rating,
        'contract': contract_or_title,
        'job': job,
        'city': city,
        'date': date,
        'comment': comment
    }
    df = df.append(new_row, ignore_index=True)

fichier.close()

/tmp/ipykernel_180/3003956925.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_180/3003956925.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_180/3003956925.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_180/3003956925.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_180/3003956925.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versio

In [64]:
df

,rating,contract,job,city,date,comment
0,3.0,Cdd,Conseillère clientèle (ancien employé),Massy (91),17 mai 2022,"Bonne équipe, dommage que la direction soit bl..."
1,5.0,Une équipe au top,Responsable (ancien employé),Falaise,28 novembre 2022,"Équipe au top, l'esprit d'équipe fait la force..."
2,4.0,agréable,Compliance Officer (ancien employé),Nanterre (92),25 novembre 2022,"bonne ambiance, bonne équipe, bon relationnel,..."
3,5.0,Très bonne expérience,Assistante administrative (ancien employé),Saint-Dié-des-Vosges (88),24 novembre 2022,"Missions variées, solidarité dans l’équipe et ..."
4,4.0,Expérience au siège,Chargée de communication (ancien employé),La Défense (92),24 novembre 2022,Première expérience professionnelle durant 3 a...
...,...,...,...,...,...,...
1847,4.0,Très agréable et formateur,stagiaire analyste financier (ancien employé),Tour Société Générale à la défense,14 décembre 2012,Stage d'analyse financière dans le secteur imm...
1848,4.0,Cool,Financial Analyst (ancien employé),"Cergy, France",4 décembre 2012,I love my co workers.\nI learnt a lot about ba...
1849,4.0,Environnement de travail,Fiscaliste Internationale en Back Office (anci...,Paris,14 novembre 2012,excellent environnement de travail.\n\nUne exp...
1850,4.0,environnement excellent,conseiller de clientèle (ancien employé),grenoble,7 novembre 2012,"environnement agréable,bonnes équipes,travail ..."


In [31]:
len(errors['rating'])

453

In [58]:
df.describe()

,rating
count,1852.000000
mean,3.762419
std,1.091063
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [59]:
#Nombre de modalités pour chacune des variables
df.nunique()

rating         5
contract    1229
job         1236
city         681
date        1276
comment     1761
dtype: int64

In [12]:
#Vérification valeurs manquantes
df.count()

rating      1852
contract    1852
job         1852
city        1852
date        1852
comment     1852
dtype: int64

In [13]:
df.isnull().sum()

rating      0
contract    0
job         0
city        0
date        0
comment     0
dtype: int64

In [69]:
print(df.dtypes)

rating      float64
contract     object
job          object
city         object
date         object
comment      object
dtype: object


In [65]:
#Création d'une copie de la base
df_new = df.copy()

In [66]:
#Modification de la colonne date pour garder uniquement l'année
for i in range(1852):
    df_new['date']=df_new['date'].replace(df_new['date'][i], df_new['date'][i].split()[-1])

In [80]:
#Suppression des codes postaux dans la colonne city 
characters = ")(1234567890"
for i in range(1852):
    for x in range(len(characters)):
        df_new['city']=df_new['city'].replace(df_new['city'][i], df_new['city'][i].replace(characters[x],"").capitalize())

In [81]:
df_new

,rating,contract,job,city,date,comment
0,3.0,Cdd,Conseillère clientèle (ancien employé),Massy,2022,"Bonne équipe, dommage que la direction soit bl..."
1,5.0,Une équipe au top,Responsable (ancien employé),Falaise,2022,"Équipe au top, l'esprit d'équipe fait la force..."
2,4.0,agréable,Compliance Officer (ancien employé),Nanterre,2022,"bonne ambiance, bonne équipe, bon relationnel,..."
3,5.0,Très bonne expérience,Assistante administrative (ancien employé),Saint-dié-des-vosges,2022,"Missions variées, solidarité dans l’équipe et ..."
4,4.0,Expérience au siège,Chargée de communication (ancien employé),Défense,2022,Première expérience professionnelle durant 3 a...
...,...,...,...,...,...,...
1847,4.0,Très agréable et formateur,stagiaire analyste financier (ancien employé),Tour société générale à la défense,2012,Stage d'analyse financière dans le secteur imm...
1848,4.0,Cool,Financial Analyst (ancien employé),"Cergy, france",2012,I love my co workers.\nI learnt a lot about ba...
1849,4.0,Environnement de travail,Fiscaliste Internationale en Back Office (anci...,Paris,2012,excellent environnement de travail.\n\nUne exp...
1850,4.0,environnement excellent,conseiller de clientèle (ancien employé),Grenoble,2012,"environnement agréable,bonnes équipes,travail ..."


In [82]:
#Nombre de modalités pour chacune des variables
df_new.nunique()

rating         5
contract    1229
job         1236
city         639
date          11
comment     1761
dtype: int64

In [87]:
import csv
df_new.to_csv(r'Database.csv', index=False, header=True)